In [ ]:
from tkinter import*
from tkinter import ttk
import PIL
from PIL import Image,ImageTk
import tkinter as tk
from tkinter import *
from tkinter import messagebox
from tkcalendar import Calendar
from tkcalendar import DateEntry
import csv
import cv2
import pandas as pd
import numpy as np
import face_recognition as face_rec
import os
import pyttsx3
import time
from datetime import  datetime
from tkinter.simpledialog import askstring
from tkinter.messagebox import showinfo



def register():
    
        
        
    def takePhoto(name):
        with open('Attendance.csv','a',newline='') as f:
            writer = csv.writer(f)   
            writer.writerow([name])
    
        df = pd.read_csv('Attendance.csv')
        df=df.sort_values(by=['Name'])
        df.fillna('Not Joined',inplace = True)
        df.to_csv('Attendance.csv',index = False)
            
            
        voice = pyttsx3.init()
        flag = False
            
        cap = cv2.VideoCapture(0)
            
        start_time = time.time()   
    
        while True:
            ret, frame = cap.read()
            cv2.imshow('img', frame)
    

            key = cv2.waitKey(1)
        
            if(flag == False and (time.time()-start_time>4)):
                
                voice.say("Press 'Enter', to capture the image")
                voice.runAndWait()
                flag=True
                #messagebox.showinfo("Message" ,"Press 'Enter' to capture the image")
                #flag = True
                

            if key==13:
                img_name = "TrainedImages/{}.jpg".format(name)
                cv2.imwrite(img_name, frame)
                voice.say("{}'s Image Succesfully Saved!".format(name))
                voice.runAndWait()
                messagebox.showinfo("Image Saved", "{}'s Image Succesfully Saved!".format(name))
                root.destroy()
                break

            if key==ord('q'):
                break


        cap.release()
        cv2.destroyAllWindows()
        
        
        
    root=tk.Tk()
    root.geometry("1390x790")
    root.title("Register New Entry")
        
    root['bg']="cyan"     
    
        
    title_lbl=Label(root,text="REGISTER NEW ENTRY",font=("times new roman",35,"bold"),bg="black",fg="yellow")
    title_lbl.place(x=0,y=15,width=1390,height=60)
        
    main_frame=Frame(root,bd=2)
    main_frame.place(x=150,y=150,width=670,height=430)
        
        
    class_student_frame=LabelFrame(main_frame,bd=2,relief=RIDGE,text="Class Student information",font=("times new roman",12,"bold"))
    class_student_frame.place(x=30,y=50,width=600,height=350)
        
        
    student_name_label=Label(class_student_frame,text="Student Name:",font=("times new roman",12,"bold"))
    student_name_label.place(x=20,y=50,width=250,height=30)
        
    studentId_entry=ttk.Entry(class_student_frame,width=20,font=("times new roman",12,"bold"))
    studentId_entry.place(x=240,y=50,width=240,height=30)
        
        
    btn_frame=Frame(class_student_frame,bd=2,relief=RIDGE,bg="white")
    btn_frame.place(x=200,y=200,width=200,height=35)
        
    save_btn=Button(btn_frame,text="Capture Image",width=21,font=("times new roman",12,"bold"),command=lambda:takePhoto(studentId_entry.get()),bg="yellow",fg="black")
    save_btn.grid(row=0,column=1,padx=2)
    
    
    
    
        
        
    root.mainloop()
    
    
    
    
    
        

        
        
        
        
def takeAttendance():
    
    def startAttendance(date,time1):
    
        df = pd.read_csv("Attendance.csv")
        df.insert(1,date,'')
        df.to_csv("Attendance.csv",index=False)
    


        engine = pyttsx3.init()

        def resize(img, size) :
            width = int(img.shape[1]*size)
            height = int(img.shape[0] * size)
            dimension = (width, height)
            return cv2.resize(img, dimension, interpolation= cv2.INTER_AREA)

        path = 'TrainedImages'
        studentImg = []
        studentName = []
        myList = os.listdir(path)
        for cl in myList :
            if(os.path.splitext(cl)[1]!='.jpg'):
                continue
            curimg = cv2.imread(f'{path}/{cl}')
            studentImg.append(curimg)
            studentName.append(os.path.splitext(cl)[0])

    
            

        def findEncoding(images) :
            imgEncodings = []
            for img in images :
                img = resize(img, 0.50)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                encodeimg = face_rec.face_encodings(img)[0]
                imgEncodings.append(encodeimg)
            return imgEncodings
        
        def MarkAttendence(name):
            df = pd.read_csv('Attendance.csv')
   
            l=df.index[df['Name']==name].tolist()
            if(df.at[l[0],date]!='P'):
            
                statment =  str(name + ", "+ " you are " + " marked present")
            
                engine.say(statment)
                engine.runAndWait()
                #messagebox.showinfo("Attendance Marked" ,"{} ,Marked Present!".format(name))
            
            df.at[l[0],date]= 'P'
            df.to_csv('Attendance.csv',index = False)
       




        EncodeList = findEncoding(studentImg)
    
        start_time = time.time() 
     
    
        vid = cv2.VideoCapture(0)
        while True :
            success, frame = vid.read()
            Smaller_frames = cv2.resize(frame, (0,0), None, 0.25, 0.25)

            facesInFrame = face_rec.face_locations(Smaller_frames)
            encodeFacesInFrame = face_rec.face_encodings(Smaller_frames, facesInFrame)

            for encodeFace, faceloc in zip(encodeFacesInFrame, facesInFrame) :
                matches = face_rec.compare_faces(EncodeList, encodeFace)
                facedis = face_rec.face_distance(EncodeList, encodeFace)
                matchIndex = np.argmin(facedis)

                if matches[matchIndex] :
                    name = studentName[matchIndex]
                    y1, x2, y2, x1 = faceloc
                    y1, x2, y2, x1 = y1*4, x2*4, y2*4, x1*4
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
                    cv2.rectangle(frame, (x1, y2-25), (x2, y2), (0, 255, 0), cv2.FILLED)
                    cv2.putText(frame, name, (x1+6, y2-6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
                    MarkAttendence(name)

            cv2.imshow('video',frame)
            key = cv2.waitKey(1)
     

            if (key==ord('q') or (time.time() - start_time) > time1):
                df=pd.read_csv('Attendance.csv')
                df.fillna('A',inplace=True)
                df.to_csv('Attendance.csv',index = False)
                break
    
        vid.release()
        cv2.destroyAllWindows()
    

        
    root = tk.Tk()
    root.title("TAKE ATTENDENCE")
    root.geometry("800x700")
     
    root['bg']="pink"    
    

    title_lbl=Label(root,text="TAKE ATTENDANCE",font=("times new roman",35,"bold"),bg="black",fg="yellow")
    title_lbl.place(x=0,y=0,width=800,height=60)


    date_label = tk.Label(root, text = 'Select Date ', font=('calibre',20, 'bold'),bg="blue",fg="yellow").place(x=75,y=70)

    cal = Calendar(root, selectmode = 'day',date_pattern='dd/mm/yyyy')
    cal=DateEntry(root,selectmode='day',date_pattern='dd/mm/yyyy')
    cal.pack(pady = 80)



    date = tk.Entry(root, font=("times new roman",20),bg="cyan", width ='26')
    date.pack(pady = 1)


    def grad_date():
        dt= cal.get_date()
        str_dt=dt.strftime("%d %B %Y") 
        date.delete(0,END)
        date.insert(0,str_dt)
    
    Button(root, text = "Select Date",font =( 'calibre 15 bold'),
           command = grad_date,bg="green",fg="yellow",width=15).pack(pady = 0)


    Duration_label = tk.Label(root, text = ' Duration(in min) ', font=('calibre',20, 'bold'),width=15,bg="blue",fg="yellow")
    Duration_label.pack(pady = 50)

    Duration_entry = tk.Entry(root, font=('calibre',15,'normal'),bg="cyan",fg="green",width=20)
    Duration_entry.pack(pady = 1)



    Button(root, text = "Start Attendance", font = 'calibre 15 bold',command = lambda:startAttendance(date.get(),int(Duration_entry.get())*60) ,width =25,bg="green",fg="yellow").place(x=250, y=500)
    Button(root, text = "Home", font = 'calibre 15 bold',  width =15,bg="green",fg="yellow",command=root.destroy).place(x=66, y=650)



    root.mainloop()
    
    
    
    
    
    
    
def view():
    
    def clear_data():
        tv1.delete(*tv1.get_children())
        return None

    root = tk.Tk()
    root.title("Attendandance Form")

    root.geometry("1400x800")
    root.pack_propagate(False)
    root.resizable(0, 0) 
    root['bg']="cyan"

    frame0 = tk.Label(root,text="ATTENDANCE FORM",font=("times new roman",40,"bold"),bg="black",fg="yellow")
    frame0.place(x=0,y=15,width=1400,height=70)


    frame1 = tk.LabelFrame(root, text="Attendannce Data")
    frame1.place(x=45,y=180,height=400, width=1300)



    tv1 = ttk.Treeview(frame1)
    tv1.place(relheight=1, relwidth=1) 

    treescrolly = tk.Scrollbar(frame1, orient="vertical", command=tv1.yview)
    treescrollx = tk.Scrollbar(frame1, orient="horizontal", command=tv1.xview) 
    tv1.configure(xscrollcommand=treescrollx.set, yscrollcommand=treescrolly.set) 
    treescrollx.pack(side="bottom", fill="x") 
    treescrolly.pack(side="right", fill="y") 


    excel_filename = r"C:\Users\bhart\OneDrive - Lovely Professional University\Documents\PythonCode\Face Recognition System\Attendance.csv"
    df = pd.read_csv(excel_filename)

    clear_data()
    
    tv1["column"] = list(df.columns)
    tv1["show"] = "headings"
    
    
    for col in tv1["columns"]:
        tv1.column(col,anchor=tk.CENTER,stretch=tk.NO,width=150)
        tv1.heading(col, text=col) 
        

    df_rows = df.to_numpy().tolist()
    for row in df_rows:
        tv1.insert("", "end", values=row) 


    
    Button(root, text = "Home", font = 'arial 15 bold', command = root.destroy, width =25,bg="red",fg="black").place(x=450, y=700)

    root.mainloop()
    
    
    
    
    
    
    
    
    
    
    
def delete():
    root = Tk()
    root.geometry('1080x680')
    root.resizable(9, 1)
    root.config(bg = '#1BB1D9')
    root.title('Delete Record in Attendance Form')


  
        
    title_lbl=Label(root,text="DELETE RECORD",font=("times new roman",35,"bold"),bg="black",fg="yellow")
    title_lbl.place(x=0,y=20,width=1080,height=60)


    def deleteUserRecord():
        name = askstring('Delete record', 'Enter the name of the student')
        df = pd.read_csv("Attendance.csv", index_col ="Name" )
        df.drop([name],inplace=True)
        df.to_csv("Attendance.csv")
        os.remove(r"C:\Users\bhart\OneDrive - Lovely Professional University\Documents\PythonCode\Face Recognition System\TrainedImages\{}.jpg".format(name))
        messagebox.showinfo("Deleted","Record deleted successfully!")
        root.destroy()
    

    def deleteDateRecord():
        date = askstring('Delete a date record', 'Enter the date in DD Month YYYY format')
        df = pd.read_csv("Attendance.csv", index_col ="Name" )
        df=df.drop([date], axis = 1)
        df.to_csv("Attendance.csv")
        messagebox.showinfo("Deleted","Record deleted successfully!")
        root.destroy()

    def clearAttendanceData():
        df = pd.read_csv("Attendance.csv" )
        df1=df['Name']
        df1.to_csv("Attendance.csv",index=False)  
        messagebox.showinfo("Deleted","Record deleted successfully!")
        root.destroy()
    
    
    def clearAllRecords():
        with open('Attendance.csv','w') as f:
            writer=csv.writer(f)
            writer.writerow(['Name'])
        dir = r"C:\Users\bhart\OneDrive - Lovely Professional University\Documents\PythonCode\Face Recognition System\TrainedImages"
        for f in os.listdir(dir):
            if(f[-4:]=='.jpg'):
                os.remove(os.path.join(dir, f))
        messagebox.showinfo("Deleted","Record deleted successfully!")
        root.destroy()

        

      
    



    Button(root, text = "Delete a user record", font = 'arial 15 bold', command = deleteUserRecord, width =25,bg="blue",fg="yellow").place(x=350, y=150)
    Button(root, text = "Delete a specific date's record", font = 'arial 15 bold', command = deleteDateRecord, width =25,bg="blue",fg="yellow").place(x=350, y=250)
    Button(root, text = "Clear Attendance Record", font = 'arial 15 bold', command = clearAttendanceData, width =25,bg="blue",fg="yellow").place(x=350, y=350)
    Button(root, text = "Clear Attendance Form", font = 'arial 15 bold', command = clearAllRecords, width =25,bg="blue",fg="yellow").place(x=350, y=450)
    Button(root, text = "Home", font = 'arial 15 bold', command = root.destroy, width =25,bg="red",fg="black").place(x=350, y=550)    





    root.mainloop()
    
    
    
        
        

root=tk.Tk()
root.geometry("1530x790+0+0")
root.title("Attendance Marking System")

img=PIL.Image.open(r"GUI Images\HomePage.jpg")
img=img.resize((1390,790),PIL.Image.ANTIALIAS)
photoimg=ImageTk.PhotoImage(img)
        
        
        
bg_img=Label(root,image=photoimg)
bg_img.place(x=0,y=0,width=1390,height=790) 
        
title_lbl=Label(bg_img,text="STUDENT ATTENDANCE MARKING SYSTEM",font=("times new roman",40,"bold"),bg="cyan",fg="red")
title_lbl.place(x=0,y=5,width=1390,height=60)
        
       
img1=PIL.Image.open(r"GUI Images\s1.png")
img1=img1.resize((220,220),PIL.Image.ANTIALIAS)
photoimg1=ImageTk.PhotoImage(img1)
        
b1=Button(bg_img,image=photoimg1,command = register,cursor="hand2")
b1.place(x=200,y=100,width=220,height=220)
        
b1_1=Button(bg_img,text="Register New Entry",cursor="hand2",font=("times new roman",15,"bold"),command=register,bg="black",fg="yellow")
b1_1.place(x=200,y=300,width=220,height=40)
        
        
        
img2=PIL.Image.open(r"GUI Images\att.png")
img2=img2.resize((220,220),PIL.Image.ANTIALIAS)
photoimg2=ImageTk.PhotoImage(img2)
        
b2=Button(bg_img,image=photoimg2,command = takeAttendance,cursor="hand2")
b2.place(x=600,y=100,width=220,height=220)
        
b2_1=Button(bg_img,text="Take Attendance",command = takeAttendance,cursor="hand2",font=("times new roman",15,"bold"),bg="black",fg="yellow")
b2_1.place(x=600,y=300,width=220,height=40)
        
        
       
        
        
img4=PIL.Image.open(r"GUI Images\td.jpg")
img4=img4.resize((220,220),PIL.Image.ANTIALIAS)
photoimg4=ImageTk.PhotoImage(img4)
        
b4=Button(bg_img,image=photoimg4,command=view,cursor="hand2")
b4.place(x=200,y=400,width=220,height=220)
        
b4_1=Button(bg_img,text="View Attendance",command = view,cursor="hand2",font=("times new roman",15,"bold"),bg="black",fg="yellow")
b4_1.place(x=200,y=600,width=220,height=40)
        
        
        
img5=PIL.Image.open(r"GUI Images\delete.jpg")
img5=img5.resize((220,220),PIL.Image.ANTIALIAS)
photoimg5=ImageTk.PhotoImage(img5)
        
b5=Button(bg_img,image=photoimg5,command = delete,cursor="hand2")
b5.place(x=600,y=400,width=220,height=220)
        
b5_1=Button(bg_img,text="Delete",command = delete,cursor="hand2",font=("times new roman",15,"bold"),bg="black",fg="yellow")
b5_1.place(x=600,y=600,width=220,height=40)
        
        

        
root.mainloop()   
        
        
        



In [4]:
#Group Members

#Anish Pathak
#Kritika Rai
#Likhitha Sree
#Mehwish Afzal Mir
#Siddharth Sen
#Tushar Bharti
#Tushar Upadhyay
#Usman Ahamad

In [ ]:
from platform import python_version

print(python_version())


In [ ]:
!pip install tk

In [ ]:
!pip install opencv-python